In [1]:
import Detection as det
import cv2
import os
import numpy as np
from PIL import Image, ImageEnhance
import shutil

# Variables for the card extraction
cardName = "KH"
counter = 0

singleImgPath = r"A:\Users\James\PycharmProjects\Blackjack\Single Images"
path = r"A:\Users\James\PycharmProjects\Blackjack\Dataset"

# Used to Extract single card for camera feed

# Get Video Stream
cap = cv2.VideoCapture(0)
while(True):
    ret, frame = cap.read()
    # Capture each frame and crop to usable area
    frame = frame[50:1050, 400:1580, :] # for 1080p
    # Perform Feature extraction to get individual cards
    edges = det.get_edges(frame)
    dilate = det.dilate_img(edges)
    contours = det.get_contours(dilate)
    cards = det.transform_cards(contours, frame)

    for card in cards:
        if counter < 1:
            cv2.imshow("card", card.warp)
            cv2.imwrite(path + "\\" + cardName + "\\" + cardName + ".jpg", card.warp)
            counter += 1

    if cv2.waitKey(1) == 27:
        break

cap.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'Detection'

In [ ]:
# Augment single card image to make multiple images with variations:
# Brightness, sharpness, colour temperature and flipped

# Augment Brightness
for root, dirs, files in os.walk(path):
    for folder in dirs:
        path = str(root) + "\\" + str(folder)
        for val in np.arange(0.6, 1.6, 0.1):
            im = Image.open(path + "\\" + folder + ".jpg")
            enhancer = ImageEnhance.Brightness(im)
            enhanced_im = enhancer.enhance(val)
            enhanced_im.save(path + "\\" + folder + "(" + str(round((val), 1)) + ").jpg")

# Flip each image
for root, dirs, files in os.walk(path):
    for folder in dirs:
        path = str(root) + "\\" + str(folder)
        for filename in os.listdir(path):
            img = Image.open(path + "\\" + filename)
            flipped = img.transpose(Image.FLIP_TOP_BOTTOM)
            flip = flipped.transpose(Image.FLIP_LEFT_RIGHT)
            flip.save(path + "\\f" + filename)

# Augment Sharpness
for root, dirs, files in os.walk(path):
    for folder in dirs:
        path = str(root) + "\\" + str(folder)
        for filename in os.listdir(path):
            for val in np.arange(0.2, 1.6, 0.2):
                im = Image.open(path + "\\" + filename)
                enhancer = ImageEnhance.Sharpness(im)
                enhanced_im = enhancer.enhance(val)
                enhanced_im.save(path + "\\" + filename.split('.')[0] + "(" + str(round((val), 1)) + ").jpg")

# Augment colour temp for each image with different brightness
# https://stackoverflow.com/questions/11884544/setting-color-temperature-for-a-given-image-like-in-photoshop - Colour temperature converter
kelvin_table = {
    4000: (255,209,163),
    4500: (255,219,186),
    5000: (255,228,206),
    5500: (255,236,224),
    6000: (255,243,239),
    6500: (255,249,253),
    7000: (245,243,255),
    7500: (235,238,255),
    8000: (227,233,255),
    8500: (220,229,255),
    9000: (214,225,255),
    9500: (208,222,255),
    10000: (204,219,255)}

def convert_temp(image, temp):
    r, g, b = kelvin_table[temp]
    matrix = ( r / 255.0, 0.0, 0.0, 0.0,
               0.0, g / 255.0, 0.0, 0.0,
               0.0, 0.0, b / 255.0, 0.0 )
    return image.convert('RGB', matrix)

for root, dirs, files in os.walk(path):
    for folder in dirs:
        path = str(root) + "\\" + str(folder)
        for filename in os.listdir(path):
            im = Image.open(path + "\\" + filename)
            for temp in kelvin_table:
                img = convert_temp(im, temp)
                img.save(path + "\\" + filename.split('.')[0] + str(temp) + ").jpg")

In [ ]:
# Split the dataset into relevant folders for training, validation and testing

# Setup Folders for data
dataset_dir = 'Dataset'
base_dir = 'Dataset_Split'

# Creating Train / Val / Test folders (One time use)
root_dir = r"A:\Users\James\PycharmProjects\Blackjack\Dataset"
suits = ["C", "S", "H", "D"]
card_values = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
classes_dir = []

# Add each card combination to cards list
for suit in suits:
    for value in card_values:
        classes_dir.append("/" + value + suit)

        
# https://stackoverflow.com/questions/53074712/how-to-split-folder-of-images-into-test-training-validation-sets-with-stratified

test_ratio = 0.10
val_ratio = 0.10

for cls in classes_dir:
    os.makedirs(root_dir +'/train' + cls)
    os.makedirs(root_dir +'/val' + cls)
    os.makedirs(root_dir +'/test' + cls)


    # Creating partitions of the data after shuffeling
    src = root_dir + cls # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)* (1 - (val_ratio + test_ratio))),
                                                               int(len(allFileNames)* (1 - test_ratio))])


    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, root_dir +'/train' + cls)

    for name in val_FileNames:
        shutil.copy(name, root_dir +'/val' + cls)

    for name in test_FileNames:
        shutil.copy(name, root_dir +'/test' + cls)
